In [1]:
%load_ext autoreload
%autoreload 2

from implemention.utils.load import *

X_train, y_train, X_val, y_val = load_train_data()
# test = load_test_data()
#X_train, y_train = X_train[:2000], y_train[:2000]
#X_val, y_val = X_val[:500], y_val[:500]

In [2]:
import keras

num_classes = 7

tar_train, tar_val = y_train, y_val
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

Using TensorFlow backend.


In [3]:
#X_pre, y_pre = X_train[:2000], y_train[:2000]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers

model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(1, 48, 48),
                 data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first')) #24

model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first')) #12

model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_first', kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first')) #6

model.add(Flatten())

# affine -> relu -> dropout * 1
model.add(Dense(1000))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))df
model.add(Dropout(0.5))

model.add(Dense(500))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(300))
model.add(BatchNormalization(axis=1, momentum=0.99, epsilon=1e-5))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

optim = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999)
#optim = keras.optimizers.SGD(lr=0.001, momentum=0.95, decay=0.0)
#optim = keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=200, epochs=25, validation_data=(X_val, y_val), shuffle=True)

Train on 24000 samples, validate on 4709 samples
Epoch 1/25
24000/24000 [==============================] - 483s 20ms/step - loss: 3.2771 - acc: 0.2018 - val_loss: 2.5901 - val_acc: 0.2729
Epoch 2/25
24000/24000 [==============================] - 417s 17ms/step - loss: 2.3888 - acc: 0.2495 - val_loss: 2.1622 - val_acc: 0.2659
Epoch 3/25
24000/24000 [==============================] - 418s 17ms/step - loss: 1.9217 - acc: 0.3435 - val_loss: 1.7970 - val_acc: 0.3619
Epoch 4/25
24000/24000 [==============================] - 425s 18ms/step - loss: 1.6733 - acc: 0.4040 - val_loss: 1.9215 - val_acc: 0.3219
Epoch 5/25
24000/24000 [==============================] - 416s 17ms/step - loss: 1.5596 - acc: 0.4358 - val_loss: 1.7182 - val_acc: 0.3763
Epoch 6/25
24000/24000 [==============================] - 420s 18ms/step - loss: 1.5001 - acc: 0.4582 - val_loss: 1.6738 - val_acc: 0.3761
Epoch 7/25
24000/24000 [==============================] - 428s 18ms/step - loss: 1.4632 - acc: 0.4753 - val_loss: 1.6

In [ ]:
import pandas as pd

test_set = load_test_data()
res = model.predict(test_set, verbose=1)
pred = np.argmax(res, axis=1)

d1 = np.arange(pred.shape[0]).reshape(-1,1)
d2 = pred.reshape(-1,1)
d = np.hstack((d1, d2))
df = pd.DataFrame(d, columns=['id', 'label'])
df.to_csv(r'submission.csv', index=False)

In [ ]:
"""
[conv->batchnorm->relu->conv->batchnorm->relu->2*2maxpool]*3->[affine->batchnorm->relu->dropout]*2->affine->softmax
Test accuracy: 0.4835421533265876

Test accuracy: 0.5185814397992995

"""